# Example Experiment
> Experiment using Repeated MNIST and BatchBALD vs BALD vs random sampling

This notebook ties everything together and runs an AL loop.

In [1]:
# experiment

#import blackhc.project.script
from tqdm.auto import tqdm

In [2]:
# experiment

import math
import time
import torch
from torch import nn as nn
from torch.nn import functional as F

from batchbald_redux import (
    active_learning,
    batchbald,
    consistent_mc_dropout,
    joint_entropy,
    repeated_mnist,
)
torch.manual_seed(0)


Let's define our Bayesian CNN model that we will use to train MNIST.

In [3]:
class BayesianCNN(consistent_mc_dropout.BayesianModule):
    def __init__(self, num_classes=10):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv1_drop = consistent_mc_dropout.ConsistentMCDropout2d()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.conv2_drop = consistent_mc_dropout.ConsistentMCDropout2d()
        self.fc1 = nn.Linear(1024, 128)
        self.fc1_drop = consistent_mc_dropout.ConsistentMCDropout()
        self.fc2 = nn.Linear(128, num_classes)

    def mc_forward_impl(self, input: torch.Tensor):
        input = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(input)), 2))
        input = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(input)), 2))
        input = input.view(-1, 1024)
        input = F.relu(self.fc1_drop(self.fc1(input)))
        input = self.fc2(input)
        input = F.log_softmax(input, dim=1)

        return input

In [4]:
from torchvision.datasets import MNIST,FashionMNIST,CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import wandb

Grab our dataset, we'll use Repeated-MNIST. We will acquire to samples for each class for our initial training set.

In [6]:
# experiment
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

for set_pool_size in [2500,5000,20000,40000]:

    train_dataset = FashionMNIST("data", train=True, download=True,transform=transform)
    test_dataset = FashionMNIST("data", train=False, download=True,transform=transform)

    train_dataset,monitor_set = random_split(
        train_dataset, [59900,100]
    )
    num_initial_samples = 20
    num_classes = 10

    initial_samples = active_learning.get_balanced_sample_indices(
        repeated_mnist.get_targets(train_dataset), num_classes=num_classes, n_per_digit=num_initial_samples / num_classes
    )

    # experiment
    max_training_samples = 300
    acquisition_batch_size = 5
    num_inference_samples = 100
    num_test_inference_samples = 5
    num_samples = 100000

    test_batch_size = 512
    batch_size = 64
    scoring_batch_size = 128
    training_iterations = 4096 * 6
    use_cuda = torch.cuda.is_available()

    print(f"use_cuda: {use_cuda}")

    device = "cuda" if use_cuda else "cpu"

    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, **kwargs)

    active_learning_data = active_learning.ActiveLearningData(train_dataset)

    # Split off the initial samples first.
    active_learning_data.acquire(initial_samples)

    # THIS REMOVES MOST OF THE POOL DATA. UNCOMMENT THIS TO TAKE ALL UNLABELLED DATA INTO ACCOUNT!
    active_learning_data.extract_dataset_from_pool(59900-set_pool_size)

    train_loader = torch.utils.data.DataLoader(
        active_learning_data.training_dataset,
        sampler=active_learning.RandomFixedLengthSampler(active_learning_data.training_dataset, training_iterations),
        batch_size=batch_size,
        **kwargs,
    )

    pool_loader = torch.utils.data.DataLoader(
        active_learning_data.pool_dataset, batch_size=scoring_batch_size, shuffle=False, **kwargs
    )
    wandb.init(project="ActiveLearning_1124_FASHION")
    wandb.run.name = f'BALD_CIFAR_1125_{set_pool_size}'
    wandb.run.save()


    # Run experiment
    test_accs = []
    test_loss = []
    added_indices = []

    pbar = tqdm(initial=len(active_learning_data.training_dataset), total=max_training_samples, desc="Training Set Size")
    epoch=0
    model = BayesianCNN(num_classes).to(device=device)
    optimizer = torch.optim.Adam(model.parameters())

    while True:

        epoch+=1
        model.train()
        train_loss=0
        # Train
        for data, target in tqdm(train_loader, desc="Training", leave=False):
            data = data.to(device=device)
            target = target.to(device=device)

            optimizer.zero_grad()

            prediction = model(data, 1).squeeze(1)
            loss = F.nll_loss(prediction, target)
            train_loss+=loss.item()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_loader)
        # Test
        loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in tqdm(test_loader, desc="Testing", leave=False):
                data = data.to(device=device)
                target = target.to(device=device)

                prediction = torch.logsumexp(model(data, num_test_inference_samples), dim=1) - math.log(
                    num_test_inference_samples
                )
                loss += F.nll_loss(prediction, target, reduction="sum")

                prediction = prediction.max(1)[1]
                correct += prediction.eq(target.view_as(prediction)).sum().item()

        loss /= len(test_loader.dataset)

        percentage_correct = 100.0 * correct / len(test_loader.dataset)



        if len(active_learning_data.training_dataset) >= max_training_samples:
            break

        # Acquire pool predictions

        acquir_start=time.time()
        N = len(active_learning_data.pool_dataset)
        logits_N_K_C = torch.empty((N, num_inference_samples, num_classes), dtype=torch.double, pin_memory=use_cuda)

        with torch.no_grad():
            model.eval()

            for i, (data, _) in enumerate(tqdm(pool_loader, desc="Evaluating Acquisition Set", leave=False)):
                data = data.to(device=device)

                lower = i * pool_loader.batch_size
                upper = min(lower + pool_loader.batch_size, N)
                logits_N_K_C[lower:upper].copy_(model(data, num_inference_samples).double(), non_blocking=True)


        with torch.no_grad():
               candidate_batch = batchbald.get_batchbald_batch(
                   logits_N_K_C, acquisition_batch_size, num_samples, dtype=torch.double, device=device
               )
    #          candidate_batch = batchbald.get_bald_batch(
    #              logits_N_K_C, acquisition_batch_size, dtype=torch.double, device=device
    #          )

        acquire_end=acquir_start-time.time()
        wandb.log({"Epoch":epoch,
                   "Num_Pool": len(active_learning_data.pool_dataset),
                   "Num_Train":len(active_learning_data.training_dataset),
                   "Test Loss": loss,
                   'Train Loss': train_loss,
                  "Test Accuracy":percentage_correct,
                   "Query Time":-acquire_end})


        targets = repeated_mnist.get_targets(active_learning_data.pool_dataset)
        dataset_indices = active_learning_data.get_dataset_indices(candidate_batch.indices)

        print("Dataset indices: ", dataset_indices)
        print("Scores: ", candidate_batch.scores)
        print("Labels: ", targets[candidate_batch.indices])

        active_learning_data.acquire(candidate_batch.indices)
        added_indices.append(dataset_indices)
        pbar.update(len(dataset_indices))

use_cuda: True


wandb: Currently logged in as: hslrock (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Dataset indices:  [10145  2957 19643 37631 39334]
Scores:  [1.2686273653606726, 2.310283010814218, 3.0890425128364063, 3.649292963078704, 4.033724475536149]
Labels:  tensor([8, 8, 5, 3, 5])


Dataset indices:  [59379 40811 58270 48947 16197]
Scores:  [1.1598164265900819, 2.1766567494504114, 3.028531435808161, 3.6479770502696125, 4.050947035502948]
Labels:  tensor([2, 6, 5, 8, 9])


...

Dataset indices:  [ 5969 12141  7140 21161 22943]
Scores:  [1.1953355311676286, 2.2660751376270962, 3.161067531172855, 3.819464419087399, 4.246071374487541]
Labels:  tensor([0, 8, 4, 9, 1])


Dataset indices:  [57970 24429 40491 21384 28977]
Scores:  [1.2147268685203265, 2.363102028579992, 3.2758655750432975, 3.9047419939209354, 4.268364198739957]
Labels:  tensor([4, 8, 0, 3, 8])


Dataset indices:  [33790 49949 39359 35495 58206]
Scores:  [1.1978587253405, 2.305762015045175, 3.2028097560874498, 3.8556138779835996, 4.244867544843541]
Labels:  tensor([3, 2, 0, 0, 2])


Dataset indices:  [38374 11330 46185 27715 36700]
Scores:  [1.2170322141882122, 2.34314838660465, 3.294231898051053, 3.9397462076144585, 4.3047741504035475]
Labels:  tensor([8, 9, 8, 1, 2])


Dataset indices:  [26763 17125 38332 13383 46294]
Scores:  [1.1905619714974156, 2.2715979654748493, 3.1850146165823032, 3.844768707763661, 4.232144284900985]
Labels:  tensor([6, 1, 8, 6, 2])


For this example, we are going to take two shortcuts that will reduce the performance:
* we discard most of the training set and only keep 20k samples; and
* we don't implement early stopping or epoch-wise training.

Instead, we always train by drawing 24576 many samples from the training set. This will overfit in the beginning and underfit later, but it still is sufficient to achieve 90% accuracy with 105 samples in the training set. 

Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Num_Pool,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
Num_Train,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Query Time,▁▁▁▁▁▁▂▂▂▁▂▁▁▁▁▂▃▃▃▅▃▄▃▄▃▄▄▄▄▄▄▄▄▄▃▄▄▄▂█
Test Accuracy,▁▂▂▃▂▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇█████
Test Loss,▆▇██▇▆█▆▅▄▅▅▅▄▅▄▄▅▄▅▄▅▄▄▄▃▃▃▂▄▃▂▂▄▃▁▂▂▂▁
Train Loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
Epoch,56
Num_Pool,9605
Num_Train,295
Query Time,34.26241


wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Dataset indices:  [13117 16516 10145 39578 30893]
Scores:  [1.3577637849649844, 1.2985060082557935, 1.268627365360672, 1.2558393956713187, 1.2239118007430252]
Labels:  tensor([2, 5, 8, 5, 8])


Dataset indices:  [49657  2957 38670 12764 31624]
Scores:  [1.2085941050498155, 1.1990055781053104, 1.1915596159850845, 1.1832860090088335, 1.1507818198196007]
Labels:  tensor([2, 8, 5, 8, 2])


...

Dataset indices:  [ 6019 47442  8476 24683 31745]
Scores:  [1.249947010969559, 1.1677006231132747, 1.145525562500342, 1.1394166292056918, 1.0867430313045965]
Labels:  tensor([5, 0, 8, 1, 0])


Dataset indices:  [11736 15318 19852 18155 28711]
Scores:  [1.09236684117062, 1.0904823901962757, 1.088929343589217, 1.0732088057345028, 1.0476247771666354]
Labels:  tensor([0, 6, 3, 0, 2])


Dataset indices:  [23146 46290 11354 47684 48404]
Scores:  [1.2358030765179262, 1.1340567151524783, 1.1015762059876657, 1.0891487976539094, 1.088999340494739]
Labels:  tensor([8, 5, 0, 3, 8])


Dataset indices:  [50023 53189 37614 18554 12136]
Scores:  [1.179881760464565, 1.0949785502683667, 1.0690284712991922, 1.065086393311453, 1.0513343967549766]
Labels:  tensor([4, 8, 2, 6, 6])


Dataset indices:  [ 5984 35561 59623 17154 34464]
Scores:  [1.1238613697512743, 1.1001181245034157, 1.0656561254528518, 1.0643599122832534, 1.0482348369257188]
Labels:  tensor([3, 8, 6, 3, 0])


In [ ]:
# experiment
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

for set_pool_size in [2500,5000,20000,40000]:

    train_dataset = FashionMNIST("data", train=True, download=True,transform=transform)
    test_dataset = FashionMNIST("data", train=False, download=True,transform=transform)

    train_dataset,monitor_set = random_split(
        train_dataset, [59900,100]
    )
    num_initial_samples = 20
    num_classes = 10

    initial_samples = active_learning.get_balanced_sample_indices(
        repeated_mnist.get_targets(train_dataset), num_classes=num_classes, n_per_digit=num_initial_samples / num_classes
    )

    # experiment
    max_training_samples = 300
    acquisition_batch_size = 5
    num_inference_samples = 100
    num_test_inference_samples = 5
    num_samples = 100000

    test_batch_size = 512
    batch_size = 64
    scoring_batch_size = 128
    training_iterations = 4096 * 6
    use_cuda = torch.cuda.is_available()

    print(f"use_cuda: {use_cuda}")

    device = "cuda" if use_cuda else "cpu"

    kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=False, **kwargs)

    active_learning_data = active_learning.ActiveLearningData(train_dataset)

    # Split off the initial samples first.
    active_learning_data.acquire(initial_samples)

    # THIS REMOVES MOST OF THE POOL DATA. UNCOMMENT THIS TO TAKE ALL UNLABELLED DATA INTO ACCOUNT!
    active_learning_data.extract_dataset_from_pool(59900-set_pool_size)

    train_loader = torch.utils.data.DataLoader(
        active_learning_data.training_dataset,
        sampler=active_learning.RandomFixedLengthSampler(active_learning_data.training_dataset, training_iterations),
        batch_size=batch_size,
        **kwargs,
    )

    pool_loader = torch.utils.data.DataLoader(
        active_learning_data.pool_dataset, batch_size=scoring_batch_size, shuffle=False, **kwargs
    )
    wandb.init(project="ActiveLearning_1124_FASHION")
    wandb.run.name = f'RBALD_CIFAR_1125_{set_pool_size}'
    wandb.run.save()


    # Run experiment
    test_accs = []
    test_loss = []
    added_indices = []

    pbar = tqdm(initial=len(active_learning_data.training_dataset), total=max_training_samples, desc="Training Set Size")
    epoch=0
    model = BayesianCNN(num_classes).to(device=device)
    optimizer = torch.optim.Adam(model.parameters())

    while True:

        epoch+=1
        model.train()
        train_loss=0
        # Train
        for data, target in tqdm(train_loader, desc="Training", leave=False):
            data = data.to(device=device)
            target = target.to(device=device)

            optimizer.zero_grad()

            prediction = model(data, 1).squeeze(1)
            loss = F.nll_loss(prediction, target)
            train_loss+=loss.item()
            loss.backward()
            optimizer.step()

        train_loss /= len(train_loader)
        # Test
        loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in tqdm(test_loader, desc="Testing", leave=False):
                data = data.to(device=device)
                target = target.to(device=device)

                prediction = torch.logsumexp(model(data, num_test_inference_samples), dim=1) - math.log(
                    num_test_inference_samples
                )
                loss += F.nll_loss(prediction, target, reduction="sum")

                prediction = prediction.max(1)[1]
                correct += prediction.eq(target.view_as(prediction)).sum().item()

        loss /= len(test_loader.dataset)

        percentage_correct = 100.0 * correct / len(test_loader.dataset)



        if len(active_learning_data.training_dataset) >= max_training_samples:
            break

        # Acquire pool predictions

        acquir_start=time.time()
        N = len(active_learning_data.pool_dataset)
        logits_N_K_C = torch.empty((N, num_inference_samples, num_classes), dtype=torch.double, pin_memory=use_cuda)

        with torch.no_grad():
            model.eval()

            for i, (data, _) in enumerate(tqdm(pool_loader, desc="Evaluating Acquisition Set", leave=False)):
                data = data.to(device=device)

                lower = i * pool_loader.batch_size
                upper = min(lower + pool_loader.batch_size, N)
                logits_N_K_C[lower:upper].copy_(model(data, num_inference_samples).double(), non_blocking=True)


        with torch.no_grad():
#                candidate_batch = batchbald.get_batchbald_batch(
#                    logits_N_K_C, acquisition_batch_size, num_samples, dtype=torch.double, device=device
#                )
             candidate_batch = batchbald.get_bald_batch(
                 logits_N_K_C, acquisition_batch_size, dtype=torch.double, device=device
             )

        acquire_end=acquir_start-time.time()
        wandb.log({"Epoch":epoch,
                   "Num_Pool": len(active_learning_data.pool_dataset),
                   "Num_Train":len(active_learning_data.training_dataset),
                   "Test Loss": loss,
                   'Train Loss': train_loss,
                  "Test Accuracy":percentage_correct,
                   "Query Time":-acquire_end})


        targets = repeated_mnist.get_targets(active_learning_data.pool_dataset)
        dataset_indices = active_learning_data.get_dataset_indices(candidate_batch.indices)

        print("Dataset indices: ", dataset_indices)
        print("Scores: ", candidate_batch.scores)
        print("Labels: ", targets[candidate_batch.indices])

        active_learning_data.acquire(candidate_batch.indices)
        added_indices.append(dataset_indices)
        pbar.update(len(dataset_indices))

use_cuda: True


Epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Num_Pool,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
Num_Train,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
Query Time,▆▆▆▆▇▆▆▆▆▆▆▆▂▂▂▂██▁▁▁▅▁▆▆▁▆▆▆▆▆▆▇▆▆▇▂▇▂▇
Test Accuracy,▁▃▄▅▅▆▆▆▆▇▇▇▆▆▆▆▆▇▇▇▇▇▇▇▇█▇█▇█▇███▇█████
Test Loss,█▇▆▆▅▅▅▅▄▄▄▄▄▅▄▅▄▄▃▃▂▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁
Train Loss,█▁▁▁▂▁▁▁▁▁▁▂▁▁▂▁▁▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
Epoch,56
Num_Pool,39705
Num_Train,295
Query Time,121.37527


wandb: wandb version 0.12.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: WARNING Calling run.save without any arguments is deprecated.Changes to attributes are automatically persisted.


Dataset indices:  [12697  6003 25062 38471 53288]
Scores:  [1.2252151897454726, 1.1875483104792552, 1.126461825707276, 1.125566788269468, 1.1253783356421825]
Labels:  tensor([4, 8, 5, 5, 2])


...

Dataset indices:  [24528 35299 59833  1998 19506]
Scores:  [1.1503706113345435, 1.1331880410207027, 1.0919414130897918, 1.072972987137765, 1.0728223699090698]
Labels:  tensor([8, 3, 0, 5, 6])


Dataset indices:  [38830 32209 24493 46624 36423]
Scores:  [1.1299462511143878, 1.0698586848124194, 1.0629828964396668, 1.0622983409698157, 1.0535814472013076]
Labels:  tensor([4, 7, 4, 5, 4])


Dataset indices:  [33953 19489 52944 32533 24105]
Scores:  [1.1849047122669802, 1.1185012111548551, 1.105868471585926, 1.0969623405283573, 1.0902579281083298]
Labels:  tensor([4, 0, 4, 8, 4])


Dataset indices:  [12616 59233 46336 38774 18351]
Scores:  [1.2358748118631133, 1.135349679376621, 1.1317860844140522, 1.082454963964877, 1.0794980030206276]
Labels:  tensor([0, 4, 0, 4, 2])


Dataset indices:  [32591 31332  9248 55584 31393]
Scores:  [1.1221567773554442, 1.1187011176801025, 1.1147474644611775, 1.0951955926022745, 1.0951884180079008]
Labels:  tensor([3, 2, 3, 0, 6])
